
# Thinking:线性规划与混合整数规划的区别是什么？
    线性规划里添加了数学公式优化，不需要满足全是整数的约束；混合整数规划无法使用数学公式对其进行优化，相当于是暴力穷举，必须满足给定的条件才可以成立。

# Thinking:针对VRP问题，常见的约束条件都有哪些？
    距离约束（如各车辆最大行驶距离，各车辆行驶距离差距最小化），路径约束（如路径先后顺序），时间窗口约束（如行驶时间最短）等

# Action2:
多辆车的路径规划 VRP：
条件：经过中国33个城市，一共4辆车，每辆车最大行驶10000公里
目标：使得每辆车的行驶里程数更接近
需要注意：
1）在VRP问题中，路径上给点赋的index和点实际的index不一样，需要使用IndexToNode方法进行转换才能得到实际的index

In [1]:
from ortools.constraint_solver import routing_enums_pb2
from ortools.constraint_solver import pywrapcp
import pandas as pd

In [46]:
# 加载数据
# 设置城市名
cities = pd.read_excel('./cities.xlsx')
city_names = cities['name'].values
city_names

array(['北京', '天津', '哈尔滨', '长春', '沈阳', '石家庄', '呼和浩特', '太原', '上海', '杭州',
       '福州', '广州', '海口', '合肥', '郑州', '南京', '济南', '兰州', '银川', '西宁', '西安',
       '拉萨', '乌鲁木齐', '成都', '重庆', '南宁', '贵阳', '昆明', '武汉', '长沙', '南昌', '香港',
       '澳门'], dtype=object)

In [123]:
# 设置数据
def create_data_model(num_vehicles,depot):
    data = {}
    data['distance_matrix'] =  pd.read_excel('./distance.xlsx',index_col=0).values
    data['num_vehicles'] = num_vehicles
    data['depot'] = depot
    return data

In [128]:
# 输出结果
def print_solution(manager, routing, solution, num_vehicles):
#     print('总行驶里程: {} 公里'.format(solution.ObjectiveValue()))
    distinct_list = [] # 记录每辆车的里程
    route_list = [] # 记录每辆车的路径规划

    for vehicle_id in range(num_vehicles):
        # 初始化距离
        route_distance = 0 
        # 针对 vehicle_id 进行规划
        route = [] 
        # 车辆起始点为每辆车
        index = routing.Start(vehicle_id)

        plan_output = '车辆的路径:\n'
        # 判断这辆车路径是否结束
        while not routing.IsEnd(index):
            # 将 manager中的index转换为distance_matrix中的index 
            index_show = manager.IndexToNode(index)
            route.append(index_show)
            previous_index = index
            # 走到下一个节点
            index = solution.Value(routing.NextVar(index))

            # 针对vehicle=0，统计previous_index到index的距离
            route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
        route_list.append(route)
        distinct_list.append(route_distance)

    print(route_list)
    print('{}公里'.format(distinct_list))
    return route_list,distinct_list

In [125]:
def add_distance_dimension(routing, transit_callback_index):
    # 添加距离约束
    dimension_name = 'Distance'
    routing.AddDimension(
        transit_callback_index,
        0,  # no slack
        10000,  # 车辆最大行驶距离
        True,  # start cumul to zero
        dimension_name)
    distance_dimension = routing.GetDimensionOrDie(dimension_name)
    # 尽量减少车辆之间的最大距离
    distance_dimension.SetGlobalSpanCostCoefficient(100)

In [126]:
def main(num_vehicles,depot):
    # 1.初始化数据，得到3个参数字典
    data = create_data_model(num_vehicles,depot)
    # 2.创建路线索引管理器，tsp_size（城市数量）, num_vehicles（车的数量）, depot（出发点：原点）
    manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                           data['num_vehicles'], data['depot'])
    
    # 3.创建路线模型 Routing Model. 
    routing = pywrapcp.RoutingModel(manager)
    
    # 4.注册距离：计算两点之间的距离，输人的是manager中两个节点index，输出distance节点之间的距离 
    def distance_callback(from_index, to_index):
        # Convert from routing variable Index to distance matrix NodeIndex.
        # 将index转换为distance_index矩阵中的节点index
        from_node = manager.IndexToNode(from_index)
        to_node = manager.IndexToNode(to_index)
        return data['distance_matrix'][from_node][to_node]
    # 注册距离函数
    transit_callback_index = routing.RegisterTransitCallback(distance_callback)
 
    # Define cost of each arc. 
    routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)
    
    # 添加距离约束
    add_distance_dimension(routing, transit_callback_index)
    
    # 5.设置参数策略，Setting first solution heuristic. 
    search_parameters = pywrapcp.DefaultRoutingSearchParameters()
    search_parameters.first_solution_strategy = (
        routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)
 
    # 6.求解路径规划
    solution = routing.SolveWithParameters(search_parameters)
    # 输出结果
    route_list,distinct_list = print_solution(manager, routing, solution,num_vehicles)
    return route_list,distinct_list

In [129]:
num_vehicles = 5
depot = 0
route_list,distinct_list = main(num_vehicles, depot)
route_list

AttributeError: 'NoneType' object has no attribute 'Value'

In [120]:
class vrp(object):
    def __init__(self, city_names=None , num_vehicles=1 ,depot=0):
        # 城市名
        self.cities = pd.read_excel('./cities.xlsx')
        self.all_city = self.cities['name'].values
        self.num_vehicles = num_vehicles
        self.depot = depot
        if city_names is None:
            self.city_names = self.all_city
        else:
            self.city_names = city_names
            self.cities = self.cities[self.cities['name'].isin(city_names)]
            
    # 设置数据
    def create_data_model(self):
        data = {}
        temp =  pd.read_excel('./distance.xlsx',index_col=0)
        temp = temp[(temp.index.isin(self.city_names))][self.city_names]
        data['distance_matrix'] = temp.values
        data['num_vehicles'] = self.num_vehicles
        data['depot'] = self.depot = depot
        return data
    
    # 获取结果
    def get_solution(self, manager, routing, solution):
        # print('总行驶里程: {} 公里'.format(solution.ObjectiveValue()))
        distinct_list = [] # 记录每辆车的里程
        route_list = [] # 记录每辆车的路径规划

        for vehicle_id in range(self.num_vehicles):
            # 初始化距离
            route_distance = 0 
            # 针对 vehicle_id 进行规划
            route = [] 
            # 车辆起始点为每辆车
            index = routing.Start(vehicle_id) 
            plan_output = '车辆的路径:\n'
            # 判断这辆车路径是否结束
            while not routing.IsEnd(index):
                # 将 manager中的index转换为distance_matrix中的index 
                index_show = manager.IndexToNode(index)
                route.append(index_show)
                previous_index = index
                # 走到下一个节点
                index = solution.Value(routing.NextVar(index))
                # 针对vehicle=0，统计previous_index到index的距离
                route_distance += routing.GetArcCostForVehicle(previous_index, index, 0)
            route_list.append(route)
            distinct_list.append(route_distance)
             
        print(route_list)
        print('{}公里'.format(distinct_list))
        return route_list,distinct_list

     # 添加距离约束
    def add_distance_dimension(self, routing, transit_callback_index):
        # 添加距离约束
        dimension_name = 'Distance'
        routing.AddDimension(
            transit_callback_index,
            0,  # no slack
            10000,  # 车辆最大行驶距离
            True,  # start cumul to zero
            dimension_name)
        distance_dimension = routing.GetDimensionOrDie(dimension_name)
        # 尽量减少车辆之间的最大距离
        distance_dimension.SetGlobalSpanCostCoefficient(100)

    def work(self):
        # 1.初始化数据，得到3个参数字典
        data = self.create_data_model()

        # 2.创建路线索引管理器，tsp_size（城市数量）, num_vehicles（车的数量）, depot（出发点：原点）
        manager = pywrapcp.RoutingIndexManager(len(data['distance_matrix']),
                                               data['num_vehicles'], data['depot'])

        # 3.创建路线模型 Routing Model. 
        routing = pywrapcp.RoutingModel(manager)

        # 4.注册距离：计算两点之间的距离，输人的是manager中两个节点index，输出distance节点之间的距离 
        def distance_callback(from_index,to_index):
            # Convert from routing variable Index to distance matrix NodeIndex.
            # 将index转换为distance_index矩阵中的节点index
            from_node = manager.IndexToNode(from_index)
            to_node = manager.IndexToNode(to_index)
            return data['distance_matrix'][from_node][to_node]
        # 注册距离函数
        transit_callback_index = routing.RegisterTransitCallback(distance_callback)

        # Define cost of each arc. 
        routing.SetArcCostEvaluatorOfAllVehicles(transit_callback_index)

        # 添加距离约束
        self.add_distance_dimension(routing, transit_callback_index)
    

        # 5.设置参数策略，Setting first solution heuristic. 
        search_parameters = pywrapcp.DefaultRoutingSearchParameters()
        search_parameters.first_solution_strategy = (
            routing_enums_pb2.FirstSolutionStrategy.PATH_CHEAPEST_ARC)

        # 6.求解路径规划
        solution = routing.SolveWithParameters(search_parameters)
        # 输出结果
        route_list,distinct_list = self.get_solution(manager, routing, solution)
        return route_list,distinct_list

In [121]:
if __name__ == '__main__':
    model = vrp(num_vehicles=5)
    route_list,distinct_list = model.work()


AttributeError: 'NoneType' object has no attribute 'Value'